## Jupiter "fuzz" testing

PR 408 Updates proseco to consider Jupiter in star and fid selection.

This notebook simulates observing Jupiter on every date for a calendar year to see if that process
"shakes out" any oddities.  It looks like at the -3C temperature used for the simulations, that the star
field including Jupiter would be "star field constrained" for parts of the simulated year, which is not
that surprising.  The Jupiter checks look correct.

In [1]:
from cxotime import CxoTime
from chandra_aca import planets
import astropy.units as u
from tqdm import tqdm
from ska_helpers.retry import retry_call

In [2]:
import sys
sys.path.insert(0, '/Users/jean/git/cheta')
sys.path.insert(0, '/Users/jean/git/proseco')
sys.path.insert(0, '/Users/jean/git/sparkles')

In [3]:
import proseco
import cheta
import sparkles

In [4]:
cheta.__version__, proseco.__version__, sparkles.__version__

('4.64.1.dev22+g9dcea5c', '5.16.3.dev47+g0174d0e', '4.29.1.dev14+g69bfcb1')

In [5]:
def mock_ocat_jupiter_for_date(date, offset_y=0, offset_z=0, detector="HRC-I"):

    jupiter_data = planets.get_planet_chandra_horizons("jupiter", date, date + 1000 * u.s)
    ra_targ = jupiter_data["ra"][0]
    dec_targ = jupiter_data["dec"][0]

    targ = {
        "obs_date": date.date,
        "detector": detector,
        "ra_targ" : ra_targ,
        "dec_targ": dec_targ,
        "offset_y": offset_y,
        "offset_z": offset_z,
        "target_name": "JUPITER",
        "focus_offset": 0,
        "sim_offset": 0,
    }

    cycle = 26

    from chandra_aca.drift import get_target_aimpoint

    chip_x, chip_y, chip_id = get_target_aimpoint(
        date=date, cycle=cycle, detector=detector,
    )

    targ.update({"chipx": chip_x, "chipy": chip_y, "chip_id": chip_id})

    from ska_sun import nominal_roll

    # Nominal roll is not quite the same targ and aca but don't care.
    targ["roll_targ"] = nominal_roll(ra_targ, dec_targ, date)

    # Set dither from defaults
    if detector.startswith("ACIS"):
        targ.update({"dither_y": 16, "dither_z": 16})
    else:
        targ.update({"dither_y": 20, "dither_z": 20})

    return targ

In [6]:
import numpy as np
from sparkles import yoshi
np.random.seed(0)

In [7]:
def print_acar(acar):
    lines = []
    lines.append("-" * 40)
    lines.append(f"obsid: {acar.obsid}")
    lines.append(f"target_name: {acar.target_name}")
    lines.append(f"P2: {-np.log10(acar.acqs.calc_p_safe())}")
    lines.append(f"guide count: {acar.guide_count}")
    lines.extend(acar.pformat())
    for msg in acar.messages:
        lines.append(f"MSG: {msg}")
    text = "\n".join(lines)
    return text

In [8]:
def run_proseco(date, obsid, offset_y=0, offset_z=0):
    yp = mock_ocat_jupiter_for_date(date, offset_y=offset_y, offset_z=offset_z, detector="HRC-I")
    pp = yoshi.convert_yoshi_to_proseco_params(**yp, obsid=obsid, t_ccd=-3, man_angle=90, duration=15000)
    aca = proseco.get_aca_catalog(**pp)
    acar = aca.get_review_table()
    acar.run_aca_review()
    return aca, acar

In [9]:
obs_cnt = 0
acas = {}
import os
out_text_file = "jupiter_select_output.txt"
if os.path.exists(out_text_file):
    os.remove(out_text_file)

# Do the last year of days
for date in tqdm(CxoTime.linspace("2024:250", "2025:250", step_max=1 * u.day)):
    obsid = -10000 - obs_cnt
    # Select a Y offset between -6 and +6 arcmin
    offset_y = np.random.uniform(-6, 6)
    # Select a Z offset between -6 and +6 arcmin
    offset_z = np.random.uniform(-6, 6)
    aca, acar = retry_call(run_proseco, [CxoTime(date), obsid],
                           {'offset_y': offset_y, 'offset_z': offset_z}, tries=3, delay=2)
    text = print_acar(acar)
    text += f"\nOffset Y: {offset_y:.2f} arcmin, Offset Z: {offset_z:.2f} arcmin\n"
    acas[obsid] = aca
    obs_cnt += 1
    with open(out_text_file, "a") as f:
        f.write(text + "\n")

import pickle
import gzip
with gzip.open("jupiter_select_acas.pkl.gz", "wb") as f:
    pickle.dump(acas, f)

  0%|          | 0/367 [00:00<?, ?it/s]

100%|██████████| 367/367 [11:09<00:00,  1.82s/it]
